In [ ]:
import keras_tuner as kt
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tcn import TCN

In [ ]:
# CNN 
####################

def build_CNN(hp):
    

    num_layers = hp.Int('num_layers', min_value=1, max_value=2, step=1)
    filters_0 = hp.Int('filters_0', min_value=12, max_value=132, step=24)
    kernel_0 = hp.Int('kernel_0', min_value=2, max_value=12, step=2)
    activation = hp.Choice("activation", ['linear',"relu", "tanh"])   #### QUITAR Y PONER CON RELU
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    pool_sizes= hp.Choice('pool', values=[0, 2])
    
    
    # define model
    inputs = tensorflow.keras.layers.Input(shape=x1[0,1:,:].shape)
   
    
    # First conv block
    x = tensorflow.keras.layers.Conv1D(filters_0, kernel_0, activation=activation, padding="same")(inputs)
    x=BatchNormalization()(x)
    
    if pool_sizes and x.shape[-2] // pool_sizes > 1:
        x = tensorflow.keras.layers.MaxPool1D(pool_size=pool_sizes)(x)
    
   # rest conv block
    if num_layers>1:
        for i in range(num_layers-1):
            x=tensorflow.keras.layers.Conv1D(filters=hp.Int(f"filters_{i+1}", min_value=12, max_value=132, step=24), kernel_size=hp.Int(f"kernel_{i+1}", min_value=2, max_value=12, step=2),
                activation=activation, padding="same")(x)
        
            if pool_sizes and x.shape[-2] // pool_sizes > 1:
                x = tensorflow.keras.layers.MaxPool1D(pool_size=pool_sizes)(x)
    
    x = tensorflow.keras.layers.Flatten()(x)
    x = tensorflow.keras.layers.Dense(y1.shape[1])(x)

    model = tensorflow.keras.Model(inputs=inputs, outputs=x)
    opt = tensorflow.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    model.compile(opt, 'mean_absolute_percentage_error')
    
    return model


# TCN 
####################

def build_TCN(hp):
    
    # Choose an optimal value for several parameters
    nb_filters = hp.Int('filters', min_value=12, max_value=132, step=24)
    kernel_size = hp.Int('kernel', min_value=2, max_value=12, step=2)
    activation = hp.Choice("activation", ['linear',"relu", "tanh"])   ###### RELU
    return_sequences = hp.Choice("sequences", [True, False])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    # Choose optimal value for dilations
    dilatCod = hp.Int('dilatCod', min_value=0, max_value=1, step=1)  
    listDilations=[[1, 2, 4, 8], [1, 2, 4, 8, 16]]
    dilations=listDilations[dilatCod]
    #dilations= [1, 2, 4, 8]
    
    # model input
    inputs = tf.keras.layers.Input(shape=x1[0,1:,:].shape)
    
    # model characteristics 
    x = TCN(
        nb_filters=nb_filters,  #
        kernel_size=kernel_size,# 
        nb_stacks=1,
        dilations=dilations,    #
        use_skip_connections=True,
        dropout_rate=0.0,
        return_sequences=return_sequences,    #
        activation=activation, #
        use_batch_norm=False,
        padding='causal',
    )(inputs)
    
    
    if return_sequences:
        x = tensorflow.keras.layers.Flatten()(x)     
    x = tensorflow.keras.layers.Dense(y1.shape[1])(x)
    model = tensorflow.keras.Model(inputs=inputs, outputs=x)
    
    # compile model
    opt = tensorflow.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    model.compile(opt, 'mean_absolute_percentage_error')
    
    return model


# LSTM 
####################

def build_LSTM(hp):
    
    # Choose an optimal value for several parameters

    recurrent_units= hp.Int('units', min_value=12, max_value=132, step=24)
    activation = hp.Choice("activation", ['linear',"relu", "tanh"])  #########  RELU
    recurrent_dropout=0.0
    return_sequences=hp.Choice("return_sequences", [True, False])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    inputs = tensorflow.keras.layers.Input(shape=x1[0,1:,:].shape)
    
    # LSTM layers
    x = tensorflow.keras.layers.LSTM(recurrent_units, activation= activation, return_sequences=return_sequences)(inputs)
    
    if return_sequences:
        x = tensorflow.keras.layers.Flatten()(x)
    
    x = tensorflow.keras.layers.Dense(y1.shape[1])(x)
   
    model = tensorflow.keras.Model(inputs=inputs, outputs=x)
    
    
    # compile model
    opt = tensorflow.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    model.compile(opt, 'mean_absolute_percentage_error')
    
    return model

